### 네이버 기사 수집
- 기사 리스트 수집 > 상세페이지 URL 만들기 > 상세 페이지에서 기사 내용 수집

In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

In [2]:
# 1. URL : 기사 리스트 수집

In [3]:
sid1, page = 101, 3
url = f"https://news.naver.com/main/mainNews.naver?sid1={sid1}&date=00:00:00&page={page}"
url

'https://news.naver.com/main/mainNews.naver?sid1=101&date=00:00:00&page=3'

In [4]:
# 2. request > response : str(json)

In [5]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3'}
response = requests.post(url, headers=headers)
response

<Response [200]>

In [6]:
# 3. str(json) > str(json) > dict, list > df

In [7]:
datas = response.json()["airsResult"]
datas = json.loads(datas)["result"][str(sid1)]
columns = ['sectionId', 'articleId', 'officeId', 'officeName', 'title', 'summary']
df = pd.DataFrame(datas)[columns]
df.tail(1)

,sectionId,articleId,officeId,officeName,title,summary
19,101,0005146438,018,이데일리,대출시장은 금융사 텃밭?…진격 나선 토뱅·카뱅,[이데일리 황병서 기자] 테크(기술)에 기반을 둔 인터넷 전문은행들이 시중은행 등 ...


In [8]:
# 4. 함수로 만들기 : 
def article_list(sid1, page):
    url = f"https://news.naver.com/main/mainNews.naver?sid1={sid1}&date=00:00:00&page={page}"
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3'}
    response = requests.post(url, headers=headers)
    datas = response.json()["airsResult"]
    datas = json.loads(datas)["result"][str(sid1)]
    columns = ['sectionId', 'articleId', 'officeId', 'officeName', 'title', 'summary']
    return pd.DataFrame(datas)[columns]

In [9]:
df = article_list(100, 2)
df.tail(1)

,sectionId,articleId,officeId,officeName,title,summary
19,100,0003666975,022,세계일보,"安, ‘무조건 완주’→‘단일화 제안’…이상돈 “딜레마에 빠졌기 때문”","이 교수 ""완주하자니 돈이 울고, 그만 두자니 '윤석열 후보 지지 선언하고 그만둬야..."


In [10]:
# article_list함수를 이용하여 100 ~ 105 카테고리, 각 카테고리별 2페이지까지 데이터 수집

In [11]:
%%time
dfs = []
for sid1 in range(100, 106):
    for page in range(1, 3):
        print(sid1, page, end=" ")
        dfs.append(article_list(sid1, page))
article_df = pd.concat(dfs, ignore_index=True)
article_df.tail(2)

100 1 100 2 101 1 101 2 102 1 102 2 103 1 103 2 104 1 104 2 105 1 105 2 Wall time: 500 ms


,sectionId,articleId,officeId,officeName,title,summary
238,105,0001950833,016,헤럴드경제,“짜장면 그릇 회수도 돈 내라” 플라스틱 줄이려다 배달비가 2배?,[헤럴드경제=박지영 기자] “짜장면 그릇 밖에 내놓기만 하면 가져가던 시절이 좋았죠...
239,105,0004018712,011,서울경제,"“한국, R&D 후츠파 정신 부족…연구 포퓰리즘 벗어나 위험 감수해야” [청론직설]",[서울경제] “미국과 이스라엘은 위험을 감수하는 것을 두려워하지 않는 풍토죠. 중국...


In [12]:
# 5. 상세 페이지 링크 생성
# https://news.naver.com/main/read.naver?mode=LSD&mid=shm
# &sid1=105&oid=092&aid=0002244690

In [13]:
article_df["link"] = "https://news.naver.com/main/read.naver?mode=LSD&mid=shm"
article_df["link"] += "&sid1=" + article_df["sectionId"]
article_df["link"] += "&oid=" + article_df["officeId"]
article_df["link"] += "&aid=" + article_df["articleId"]
link = article_df.loc[200, "link"]
print(link)

https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=023&aid=0003672832


In [14]:
# 6. request > response : str(html)

In [15]:
response = requests.get(link, headers=headers)
response

<Response [200]>

In [16]:
# 7. str(html) > css-selector > text(기사내용) > 기사 리스트 데이터 프레임에 적용

In [17]:
dom = BeautifulSoup(response.text, "html.parser")
content = dom.select_one("#articleBodyContents").text.strip()

# 환경에 따라서 아래의 코드를 실행
# content = dom.select_one("#articleBodyContents").text.strip().split("{}")[1]

content[:100]

'[사이언스카페]동료의 뼈를 살피는 아프리카 코끼리. 해마다 코끼리 5만 마리가 상아 때문에 희생된다./Karl Ammann\t\t\t\t\t\t\t\t\t\t연쇄 살인범 수사에 결정적인 역할을 했던'

In [18]:
def article_content(link):
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.text, "html.parser")
    return dom.select_one("#articleBodyContents").text.strip()

    # 환경에 따라서 아래의 코드를 실행
    # return dom.select_one("#articleBodyContents").text.strip().split("{}")[1]

In [19]:
%%time
article_df["content"] = article_df["link"].apply(article_content)

Wall time: 51.3 s


In [20]:
%%time
article_df["content"] = ""
for idx, data in article_df.iterrows():
    print(idx, end=" ")
    article_df.loc[idx, "content"] = article_content(data.link)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 Wall time: 49.2 s


In [21]:
# 모듈로 만들기

In [22]:
%%writefile naver.py
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3'}

def article_list(sid1, page):
    url = f"https://news.naver.com/main/mainNews.naver?sid1={sid1}&date=00:00:00&page={page}"
    response = requests.post(url, headers=headers)
    datas = response.json()["airsResult"]
    datas = json.loads(datas)["result"][str(sid1)]
    columns = ['sectionId', 'articleId', 'officeId', 'officeName', 'title', 'summary']
    return pd.DataFrame(datas)[columns]

def article_content(link):
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.text, "html.parser")
    return dom.select_one("#articleBodyContents").text.strip()

    # 환경에 따라서 아래의 코드를 실행
    # return dom.select_one("#articleBodyContents").text.strip().split("{}")[1]

def crawling(start, end, filename="naver_article.csv"):
    
    print("crawling article list ...")
    dfs = []
    for sid1 in range(100, 106):
        for page in range(start, end + 1):
            print(sid1, page, end=" ")
            dfs.append(article_list(sid1, page))
    article_df = pd.concat(dfs, ignore_index=True)
    
    print("\n\nmaking link ...")
    article_df["link"] = "https://news.naver.com/main/read.naver?mode=LSD&mid=shm"
    article_df["link"] += "&sid1=" + article_df["sectionId"]
    article_df["link"] += "&oid=" + article_df["officeId"]
    article_df["link"] += "&aid=" + article_df["articleId"]
    
    print("\ncrawling content ...")
    article_df["content"] = article_df["link"].apply(article_content)
    
    article_df.to_csv(filename, index=False)

Writing naver.py


In [23]:
# filename = datetime.now().strftime("%Y%m%d_%H%M%S")
# crawling(1, 1, f"{filename}.csv")

In [24]:
import naver
from datetime import datetime

In [ ]:
filename = datetime.now().strftime("%Y%m%d_%H%M%S")
naver.crawling(1, 1, f"{filename}.csv")

crawling article list ...
100 1 101 1 102 1 103 1 104 1 105 1 

making link ...

crawling content ...


In [ ]:
%ls

In [ ]:
df = pd.read_csv("20220210_134040.csv")
df.tail(2)